In [1]:
# Importing library
import numpy as np
import serial
import array
import time
import binascii
import sys
import numpy
from matplotlib import pyplot as plt
import struct


class HL_emulator:
    def __init__(self, port="", speed_rate=921600):

        self.port = port
        self.speed_rate = speed_rate

    def serial_start(self):
        # If you have problems please check that library pyserial is installed and lib serial is not installed
        # Also you need USB-COM driver cp2102
        # Probably you don't know the number of serial com port in you PC, then you need to push win+R and write command "devmgmt.msc". After that find 'com_port' tab and silicon lab devices.
        ser = serial.Serial(
            self.port,
            self.speed_rate,
            timeout=5,
            xonxoff=False,
            rtscts=False,
            dsrdtr=False,
        )
        self.ser = ser
        ser.setRTS(False)
        ser.setDTR(False)
        print("Port connected")

    def serial_close(self):
        self.ser.close()
        print("Port disconnected")

    def get_serial(self):
        return self.ser

    def serial_is_open(self):
        if self.ser.isOpen()==True:
            print("Serial is opening")
        else:
            print("Serial is close")
        return self.ser.isOpen()
        

class STM_API:

    def __init__(self, ser):
        self.ser = ser

    def CRC_Calc(self, packet: bytearray):
        crc = 0
        for ik in range(2, packet[2] + 2):
            crc += packet[ik]
        crc = ~crc
        crc &= 0xFF
        return bytearray([crc])

    def Send_Packet(self, cmd: bytes, data: bytearray, data_length: bytes):
        packet = bytearray([0xFF, 0xFF, data_length + 2, cmd]) + data
        packet += self.CRC_Calc(packet)
        self.ser.flushInput()
        self.ser.write(packet)
        return 0

    def robot_set_coord(self, x: float, y: float, phi: float):
        cmd = 0x06
        data = bytearray(self.struct.pack("f", x))
        data += bytearray(self.struct.pack("f", y))
        data += bytearray(self.struct.pack("f", phi))
        data_length = 12
        self.Send_Packet(cmd, data, data_length)

    def robot_set_speed(self, dx: float, dy: float, dphi: float):
        cmd = 0x04
        data = bytearray(struct.pack("f", dx))
        data += bytearray(struct.pack("f", dy))
        data += bytearray(struct.pack("f", dphi))
        data_length = 12
        self.Send_Packet(cmd, data, data_length)

    def robot_get_speed(self):
        cmd = 0x05
        data = bytearray()
        data_length = 0
        self.Send_Packet(cmd, data, data_length)
        rx_packet = self.ser.read(17)
        speed = struct.unpack("f", rx_packet[4:8])
        speed += struct.unpack("f", rx_packet[8:12])
        speed += struct.unpack("f", rx_packet[12:16])
        return speed

    def robot_get_coord(self):
        cmd = 0x07
        data = bytearray()
        data_length = 0
        self.Send_Packet(cmd, data, data_length)
        rx_packet = self.ser.read(17)
        coord = self.struct.unpack("f", rx_packet[4:8])
        coord += self.struct.unpack("f", rx_packet[8:12])
        coord += self.struct.unpack("f", rx_packet[12:16])
        return coord

    def get_angle(self,ID):
        cmd = 0x08
        data = bytearray([ID])
        data_length = 1
        self.Send_Packet(cmd,data,data_length)

        rx_packet = self.ser.read(7)
        angle = struct.unpack('h', rx_packet[4:6])
        print(angle)
    
    def set_angle(self, ID, angle):
        cmd = 0x03
        data = bytearray([ID,angle&0xFF,(angle>>8)&0xFF])
        data_length = 3
        self.Send_Packet(cmd,data,data_length)
        time.sleep(0.01)
    
    def grub_pile(self, side):
        cmd = 0x09
        

        data = bytearray([side])
        data_length = 1

        self.Send_Packet(cmd,data,data_length)
    
    def release_pile(self,side):
        cmd = 0x0A
        

        data = bytearray([side])
        data_length = 1

        self.Send_Packet(cmd,data,data_length)
    
    def get_grub_status(self,side):
        cmd = 0x0B
        data = bytearray(struct.pack('i', side))
        data_length = 4
        self.Send_Packet(cmd,data,data_length)
        rx_packet = self.ser.read(6)
        status = rx_packet[4]
        return status

    def get_release_status(self,side):
        cmd = 0x0D
        data = bytearray(struct.pack('i', side))
        data_length = 4
        self.Send_Packet(cmd,data,data_length)
        rx_packet = self.ser.read(6)
        status = rx_packet[4]
        return status

    def set_wheel_mode(self, ID):
        cmd = 0x0F
        data = bytearray([ID])
        data_length = 1
        self.Send_Packet(cmd, data, data_length)

    def set_joint_mode(self, ID):
        cmd = 0x10
        data = bytearray([ID])
        data_length = 1
        self.Send_Packet(cmd, data, data_length)
    
    def set_speed(self, ID, speed):
        cmd = 0x11
        # data = bytearray([ID, *speed.to_bytes(2, 'little')])
        data =bytearray([ID])
        data += bytearray(struct.pack('h', speed))
        data_length = 3
        self.Send_Packet(cmd, data, data_length)

    def step_angle(self, ID, angle):
        cmd = 0x13
        data =bytearray([ID])
        data += bytearray(struct.pack('i', angle))
        data_length = 5
        self.Send_Packet(cmd, data, data_length)

    def stop_stepper(self, ID):
        cmd = 0x14
        data =bytearray([ID])
        data_length = 1
        self.Send_Packet(cmd, data, data_length)

    def set_pwm(self, ID, speed):
        cmd = 0x12
        data = bytearray([ID, *speed.to_bytes(2, 'little')])
        data_length = 3
        self.Send_Packet(cmd, data, data_length)

    def calibrate(self, ID):
        self.set_pwm(ID, 100)
        time.sleep(3)
        self.set_pwm(ID, 200)
        time.sleep(3)
        self.set_pwm(ID, 100)

    def calibrateNewEmp(self, ID):
        self.set_pwm(ID, 200)
        time.sleep(3)
        self.set_pwm(ID, 100)

    def robot_set_speed_e(self, dx: float, dy: float, dphi: float):
        cmd = 0x15
        data = bytearray(struct.pack("f", dx))
        data += bytearray(struct.pack("f", dy))
        data += bytearray(struct.pack("f", dphi))
        data_length = 12
        self.Send_Packet(cmd, data, data_length)
    def encoder_check(self, ID):
        cmd = 0x18
        data = bytearray(struct.pack("B", ID))
        data_length = 1
        if ID == 22:
            self.Send_Packet(cmd, data, data_length)
            rx_packet = self.ser.read(14)
            print(struct.unpack("i", rx_packet[5:9]))
            print(struct.unpack("i", rx_packet[9:13]))
        else:
            self.Send_Packet(cmd, data, data_length)
            rx_packet = self.ser.read(10)
            print("ID = ")
            print(rx_packet[4])
            if rx_packet[4] == 2:
                print(struct.unpack("i", rx_packet[5:9]))
            else:
                print(struct.unpack("i", rx_packet[5:9]))

        
    def DC(self, ID, dir, speed):
        cmd = 0x16
        data = bytearray(struct.pack("B", ID))
        data += bytearray(struct.pack("B", dir))
        data += bytearray(struct.pack("H", speed))
        data_length = 4
        self.Send_Packet(cmd, data, data_length)
    def setPosition(self, ID, angle):
        cmd = 0x17
        angle_ticks = angle
        data = bytearray(struct.pack("B", ID))
        data += bytearray(struct.pack("i", angle_ticks))
        data_length = 5
        self.Send_Packet(cmd, data, data_length)
    def setMaxonPosition(self, ID, angle):
        cmd = 0x17
        angle_ticks = angle
        data = bytearray(struct.pack("B", ID))
        data += bytearray(struct.pack("i", angle_ticks))
        data_length = 5
        self.Send_Packet(cmd, data, data_length)
    def sort_golden_bot(self):
        cmd = 0x18
        data = bytearray(struct.pack("B", 1))
        data_length = 1
        self.Send_Packet(cmd, data, data_length)
    def sort(self):
        cmd = 0x1A
        data = bytearray()
        data_length = 0
        self.Send_Packet(cmd, data, data_length)

    def sort_status(self):
        cmd = 0x1B
        data = bytearray()
        data_length = 0
        self.Send_Packet(cmd, data, data_length)
        rx_packet = self.ser.read(6)
        status = rx_packet[4]
        print(status)
    def servo_angle(self, ID, angle):
        cmd = 0x12
        data = bytearray([ID])
        angle_trans = int(7 / 6 * angle) + 45
        data += bytearray(struct.pack("H", angle_trans))
        data_length = 3
        self.Send_Packet(cmd, data, data_length)
    def zeroPosition(self):
        cmd = 0x1D
        data = bytearray()
        data_length = 0
        self.Send_Packet(cmd, data, data_length)
    def optimalSort(self):
        cmd = 0x1C
        data = bytearray()
        data_length = 0
        self.Send_Packet(cmd, data, data_length)
    def switchEmp(self, sw):
        cmd = 0x1F
        data = bytearray([sw])
        data_length = 1
        self.Send_Packet(cmd, data, data_length)
    def gripperCalibration(self):
        cmd = 0x20
        data = bytearray()
        data_length = 0
        self.Send_Packet(cmd, data, data_length)
    def encoder_reset(self, ID):
        idies = [10, 11, 12]
        for id in ID:
            self.encoder_check(idies[id])
    def impel_test(self, speed):
        for j in range(10):
            for i in range(2):
                self.set_pwm(7, 100)
                time.sleep(5)
                self.switchEmp(0)
                time.sleep(1)
                self.set_pwm(7, speed + 100)
                time.sleep(12)

                self.set_pwm(7, 100)
                time.sleep(5)
                self.switchEmp(1)
                time.sleep(1)
                self.set_pwm(7, speed + 100)
                time.sleep(12)
            self.set_pwm(7, 100)
            time.sleep(2*60)
    def game_start(self):
        cmd = 0x21
        data = bytearray()
        data_length = 0
        self.Send_Packet(cmd, data, data_length)
        rx_packet = self.ser.read(6)
        status = rx_packet[4]
        print(status)
    



In [100]:
#vova_em='/dev/cu.SLAB_USBtoUART'
ser_em_desk="COM4"


emulator=HL_emulator(port=vova_em)q
emulator.serial_start()

emulator.serial_is_open()

ST=STM_API(ser=emulator.get_serial())

Port connected
Serial is opening


In [22]:
ST.encoder_check(0)
ST.encoder_check(1)

ID = 
0
(-580,)
ID = 
1
(-1158,)


In [122]:
ST.game_start()

0


In [81]:
ST.encoder_check(2)

ID = 
2
(0,)


In [469]:
ST.servo_angle(7, 40)

In [111]:
ST.setPosition(2, 200)

In [108]:
ST.setPosition(1, 0)

In [103]:
ST.setPosition(2,354)

In [1171]:
ST.switchEmp(0)

In [298]:
ST.set_angle(2, 22)

In [19]:
ST.gripperCalibration()

In [23]:
ST.zeroPosition()

In [897]:
ST.set_pwm(6, 190)

In [898]:
ST.set_pwm(6, 100)

In [831]:
ST.set_pwm(7, 200)

In [832]:
ST.set_pwm(7, 200)
time.sleep(2)
ST.set_pwm(7, 100)

In [145]:
ST.get_angle(5)

(129,)


In [159]:
ST.encoder_check(2)

ID = 
2
(0,)


In [935]:
ST.grub_pile(1)
ST.grub_pile(2)
ST.grub_pile(3)

In [936]:
ST.release_pile(1)
ST.release_pile(2)
ST.release_pile(3)

In [26]:
ST.zeroPosition()


In [20]:
ST.encoder_reset([0,1,2])

ID = 
10
(-10000160,)
ID = 
11
(-10000160,)
ID = 
12
(-10000160,)


In [25]:
ST.optimalSort()

In [916]:
ST.grub_pile(1)
ST.grub_pile(2)
ST.grub_pile(3)

In [917]:
ST.release_pile(1)
ST.release_pile(2)
ST.release_pile(3)

In [939]:
# time.sleep(5)
ST.sort()

In [673]:
ST.sort_status()

1


In [84]:
ST.encoder_check(2)

ID = 
2
(2,)


In [931]:
ST.setPosition(2, 8)

In [649]:
ST.setPosition(2, 0)

In [984]:
ST.encoder_check(2)

error: (6, 'Device not configured')

In [295]:
ST.switchEmp(0) # zero - clockwise - suck

In [36]:
ST.set_pwm(7, 100)
# ST.set_pwm(1, 150)
# ST.calibrateNewEmp(1)

In [12]:
time.sleep(2)
ST.set_pwm(7, 200)
time.sleep(2)
ST.set_pwm(7, 100)


In [46]:
# ST.set_pwm(1, 100)
ST.set_pwm(7, 100)

In [825]:
ST.zeroPosition()

In [491]:
ST.sort()

In [42]:
ST.optimalSort()

In [84]:
ST.encoder_check(66)

(0,)


In [3]:
ST.servo_angle(7, 0)

In [ ]:
ST.set_pwm(7, 255)

In [107]:
ST.setPosition(2, 465)

In [93]:
ST.setPosition(1, 420)

In [123]:
ST.setPosition(1, 0)

In [146]:
ST.setPosition(2, 0)

In [201]:
for i in range(3):
    ST.grub_pile(i + 1)

In [492]:
for i in range(3):
    ST.release_pile(i + 1)

In [203]:
ST.sort()

In [615]:
ST.zeroPosition()

In [52]:
ST.setPosition(1, 2000030000)

error: 'i' format requires -2147483648 <= number <= 2147483647

In [106]:
ST.encoder_check(2)

ID = 
2
(32000,)


In [103]:
ST.encoder_check(2)

ID = 
2


error: unpack requires a buffer of 4 bytes

In [165]:
ST.setPosition(2, 0)

In [492]:
ST.set_angle(3, 30)

In [99]:
ST.DC(0,1,0)

In [116]:
ST.sort_golden_bot()

In [97]:
ST.set_angle(10, 280)

In [79]:
now = time.time_ns()
ST.robot_set_speed(0, 0, 2)
while(1):
    speed = ST.robot_get_speed()[2]
    if abs(speed-2) < 0.01:
        print(speed)
        print(time.time_ns() - now)
        ST.robot_set_speed(0, 0, 0)
        break

1.9916267395019531
129924000


In [ ]:

now = time.time_ns()
ST.robot_set_speed(0, 0, 2)
while(1):
    speed = ST.robot_get_speed()[2]
    if abs(speed-2) < 0.01:
        print(speed)
        print(time.time_ns() - now)
        ST.robot_set_speed(0, 0, 0)
        break

In [86]:
ST.robot_set_speed(0,0,0)

In [61]:
emulator.serial_close()

Port disconnected


In [4]:
ST.step_angle(0, 0)

In [69]:
ST.stop_stepper(3)

In [102]:
ST.robot_set_speed_e(0, 0, 7.85)
time.sleep(2.5)
print(ST.robot_get_speed()[2])
ST.robot_set_speed_e(0, 0, 0)
now = time.time_ns()
while(1):
    speed = ST.robot_get_speed()
    if abs(speed[0]) < 0.001 and abs(speed[1]) < 0.001 and abs(speed[2]) < 0.001:
        # print(speed)
        print(time.time_ns() - now)
        break
    

7.755181312561035
1445055000


In [10]:
ST.get_angle(11)

(300,)


In [9]:
ST.set_joint_mode(11)

In [48]:
# ST.set_wheel_mode(11)
for i in range(3):
    ST.set_speed(11, -1023)
    time.sleep(1.43)
    ST.set_speed(11, 0)
    time.sleep(1)
    ST.set_speed(11, 1023)
    time.sleep(1.43)
    ST.set_speed(11, 0)
    time.sleep(1)

In [130]:
int(5 / 360.0 * 1530)


21

In [257]:
ST.setPosition(1, 0)

In [361]:
ST.sort()

In [523]:
for i in range(3):
    ST.grub_pile(i + 1)

In [524]:
ST.setPosition(2, 0)

In [445]:
for i in range(3):
    ST.release_pile(i + 1)

In [526]:
ST.setPosition(2, 100)

In [447]:
ST.set_angle(4, 151)
ST.set_angle(5, 151)
ST.set_angle(6, 151)

In [528]:
ST.setPosition(2, 130) # up position = 460, down = 0, p - up, sort = 118

In [529]:
for i in range(3):
    ST.grub_pile(i + 1)

In [530]:
ST.setPosition(2, 470)

In [531]:
ST.setPosition(0, -420) # down gripper, p - counter-clockwise, step 430 

In [532]:
ST.setPosition(1, 840) # up gripper, p - clockwise

In [533]:
ST.setPosition(2, 130)

In [534]:
for i in range(3):
    ST.release_pile(i + 1)

In [535]:
ST.setPosition(2, 0)

In [78]:
ST.set_angle(4, 151)
ST.set_angle(6, 151)

ST.set_angle(1, 45)
ST.set_angle(3, 45)

In [537]:
ST.set_angle(4, 135)
ST.set_angle(6, 135)

ST.set_angle(1, 25)
ST.set_angle(3, 25)

ST.set_angle(2, 45)
ST.set_angle(5, 151)

In [538]:
ST.setPosition(2, 100)

In [539]:
ST.set_angle(4, 151)
ST.set_angle(6, 151)

In [540]:
ST.setPosition(2, 130)

In [541]:
ST.set_angle(1, 45)
ST.set_angle(3, 45)

In [543]:
ST.setPosition(2, 470)

In [544]:
ST.setPosition(1, 420)

In [545]:
ST.setPosition(0, 0)

In [546]:
ST.set_angle(3, 25)
ST.set_angle(6, 135)

In [547]:
ST.setPosition(0, 420)

In [548]:
ST.set_angle(1, 25)

In [549]:
ST.set_angle(1, 30)
ST.set_angle(2, 30)
ST.set_angle(3, 30)

ST.set_angle(5, 140)
ST.set_angle(6, 140)

In [550]:
ST.setPosition(2, 130)

In [551]:
ST.set_angle(2, 23)
ST.set_angle(3, 23)

In [552]:
ST.setPosition(2, 0)

In [553]:
ST.set_angle(2, 45)
ST.set_angle(3, 45)

In [554]:
ST.setPosition(2, 470)

In [555]:
ST.setPosition(1, 0)

In [556]:
ST.set_angle(4, 135)

In [557]:
ST.setPosition(0, 0)

In [558]:
ST.set_angle(3, 25)

In [559]:
ST.setPosition(0, -420)

In [560]:
ST.set_angle(2, 25)

In [561]:
ST.setPosition(0, 0)

In [562]:
ST.setPosition(1, 0)

In [566]:
ST.set_angle(0, 10)

In [570]:
# ST.setPosition(1, 0)
ST.setPosition(2, -30)

In [565]:
ST.setPosition(2, -465)

In [23]:
ST.setPosition(2, 0)

In [134]:
ST.setPosition(2, 465)

In [104]:
ST.sort(2)

ID = 
2
(25490,)


In [116]:
ST.DC(2, 0, 100)

In [124]:
ST.set_angle(4, 130)

hui 7


In [11]:
print('jj %d' % np.array([5, 6]))

TypeError: %d format: a number is required, not numpy.ndarray

In [41]:
ST.setPosition(2, 400)

In [48]:
ST.encoder_check(1)

ID = 
1
(41014,)


In [69]:
ST.setMaxonPosition(1, 2_000_002_000)

In [68]:
ST.encoder_check(1)

ID = 
1
(2000032000,)


array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
        67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
        78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
        89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
       100.])